In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 12}
----------
[[13815  6134]
 [  802 19156]]
----------
              precision    recall  f1-score   support

           0       0.95      0.69      0.80     19949
           1       0.76      0.96      0.85     19958

    accuracy                           0.83     39907
   macro avg       0.85      0.83      0.82     39907
weighted avg       0.85      0.83      0.82     39907

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.34856749, 0.4024241 , 0.47772205, 0.57446456, 0.54404533,
        0.58443606, 0.65824008, 0.72755611, 0.7310456 , 0.78689563,
        0.840253  , 0.85920191, 0.86219323, 0.84922981, 1.10155499,
        0.51711679, 0.57994914, 0.68217611, 0.76096523, 0.78440106,
        0.84623754, 0.94746876, 1.01877654, 1.14045036, 1.16189265,
        1.24317467, 1.19330931, 1.16887403, 1.10953295, 1.13197231]),
 'std_fit_time': array([9.47475433e-03, 1.49655342e-03, 1.19209290e-07, 3.09176445e-02,
        4.48668003e-03, 9.97424126e-04, 7.15255737e-07, 1.14706755e-02,
        1.39631033e-02, 2.39359140e-02, 1.14690065e-02, 2.49290466e-03,
        4.48811054e-03, 1.24664307e-02, 4.48739529e-03, 2.04451084e-02,
        9.47403908e-03, 1.09703541e-02, 1.09711885e-02, 6.48272038e-03,
        5.48589230e-03, 9.97471809e-03, 4.48715687e-03, 5.48565388e-03,
        4.18884754e-02, 2.94226408e-02, 1.74534321e-02, 3.78980637e-02,
        2.49278545e-03, 4.48801517e-02]),
 'mean_scor